In [ ]:
from datetime import datetime

from flask import Flask, redirect, render_template, url_for, request, session, flash
from flask_sqlalchemy import SQLAlchemy 
from flask_wtf import FlaskForm, CsrfProtect
from wtforms import StringField, IntegerField, validators
from wtforms.validators import ValidationError, DataRequired, Length, InputRequired, AnyOf, Regexp
from flask_bootstrap import Bootstrap

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///users.sqlite3'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] =  False
app.secret_key = "hello"

db = SQLAlchemy(app)
Bootstrap(app)

class users(db.Model):
    _id = db.Column("id", db.Integer, primary_key=True)
    item = db.Column(db.String(255))
    quantity = db.Column(db.Integer)
    
    def __init__(self, item, quantity):
        self.item = item
        self.quantity = quantity
        
class LoginForm(FlaskForm):
    username = StringField('Item', [validators.Length(min=3, max=255), validators.Regexp('[0-9A-Za-z_]+'), validators.DataRequired('Key an Item')], default='Apple')
    numbers = StringField('Numbers', [validators.Length(min=0, max=255), validators.Regexp('[0-999]+'), validators.DataRequired('Min is 0, Max is 999')], default='1')
    
@app.route('/', methods=["POST","GET"])
def login():
    form = LoginForm()
    form.validate()
    quan = None
    item = None
    if request.method == "POST":
        quan = form.numbers.data
        item = form.username.data
        session["quan"] = quan
        session["item"] = item
        usr = users(item, quan)
        
        found_user =users.query.filter_by(item =item).first()
        if found_user:
            found_user.quantity = int(quan) + int(found_user.quantity)
            db.session.commit()
        
        else:
            usr = users(item, quan)
            db.session.add(usr)
            db.session.commit()  

        flash(f"You add {form.numbers.data} {form.username.data} in warehouse" , "info")
        return render_template("login.html", form=form, values=users.query.all())
    else :
        return render_template("login.html", form=form, values=users.query.all())


if __name__ == "__main__":
    db.create_all()
    app.run()


 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [27/Mar/2021 10:51:13] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [27/Mar/2021 10:51:15] "POST / HTTP/1.1" 200 -
